In [11]:
from bs4 import BeautifulSoup
import requests
import csv
import time

In [32]:
FIELDS = ['Name', 'Division', 'Season', 'AvAge', 'GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 
          'GA', 'SRS', 'SOS', 'GF/G', 'GA/G', 'PP', 'PPO', 'PP%', 'PPA', 'PPOA', 
          'PK%', 'SH', 'SHA', 'S', 'S%', 'SA', 'SV%', 'PDO', 'SO', 'S%', 
          'SV%', 'PDO', 'CF', 'CA', 'CF%', 'xGF', 'xGA', 'aGF', 'aGA', 'axDiff', 
          'SCF', 'SCA', 'SCF%', 'HDF', 'HDA', 'HDF%', 'HDGF', 'HDC%', 'HDGA', 'HDCO%']

In [2]:
START_SITE = 'https://www.hockey-reference.com/leagues/NHL_2024.html'
start_response = requests.get(START_SITE)
start_soup = BeautifulSoup(start_response.text, 'html.parser')

In [3]:
eastern_conference_teams = start_soup.select(selector='table#standings_EAS tr th a')
eastern_conference_team_links = [team.get('href') for team in eastern_conference_teams]
western_conference_teams = start_soup.select(selector='table#standings_WES tr th a')
western_conference_team_links = [team.get('href') for team in western_conference_teams]
team_links = eastern_conference_team_links + western_conference_team_links

In [27]:
current_link = team_links[0]

new_site = 'https://www.hockey-reference.com' + current_link
response = requests.get(new_site)
soup = BeautifulSoup(response.text, 'html.parser')
team_name = soup.select(selector='h1 span')[1].text
division = soup.select(selector='div#meta div p')[2].text.split(' ')[7][4:]
print(team_name)
print(division)

Florida Panthers
Atlantic


In [18]:
team_stats = soup.select(selector='table#team_stats tr')[1]
for stat in team_stats:
    statistic = stat.get('data-stat')
    if statistic == 'team_name':
        continue
    value = stat.text
    print(statistic, value)
team_analytics = soup.select(selector='table#team_stats_adv tr')[1]
for analytic in team_analytics:
    statistic = analytic.get('data-stat')
    if statistic == 'team_name':
        continue
    value = analytic.text
    print(statistic, value)

average_age 29.5
games 82
wins 52
losses 24
losses_ot 6
points 110
points_pct .671
goals 265
goals_against 198
srs 0.81
sos -0.02
goals_for_per_game 3.23
goals_against_per_game 2.41
goals_pp 63
chances_pp 268
power_play_pct 23.51
opp_goals_pp 51
opp_chances_pp 291
pen_kill_pct 82.47
goals_sh 8
opp_goals_sh 9
shots 2764
shot_pct 9.6
shots_against 2279
save_pct .913
pdo 100.6
shutouts 8
shot_pct_5on5 7.2
sv_pct_5on5 .935
pdo 100.6
corsi_for_5on5 4212
corsi_against_5on5 3288
corsi_pct_5on5 56.2
exp_on_goals_for 166.2
exp_on_goals_against 144.8
actual_goals 156
actual_goals_against 119
actual_expected_diff 16
sc_for 1889
sc_against 1550
sc_for_pct 54.9
hdsc_for 673
hdsc_against 545
hdsc_for_pct 55.3
hdscgoal_for 51
hdsc_shot_pct 7.0
hdscgoal_against 57
hdsc_opp_shot_pct 9.5


In [33]:
print(soup.select_one(selector='h1 span').text)

2023-24


In [34]:
with open('team_data.csv', 'w', newline='') as csvfile:
    fieldnames = FIELDS
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for link in team_links:
        data_dict = {}
        new_site = 'https://www.hockey-reference.com' + link
        response = requests.get(new_site)
        soup = BeautifulSoup(response.text, 'html.parser')
        team_name = soup.select(selector='h1 span')[1].text
        division = soup.select(selector='div#meta div p')[2].text.split(' ')[7][4:]
        season = soup.select(selector='h1 span')[0].text
        data_dict['Name'] = team_name
        data_dict['Division'] = division
        data_dict['Season'] = season
        team_stats = soup.select(selector='table#team_stats tr')[1]
        team_analytics = soup.select(selector='table#team_stats_adv tr')[1]
        for stat in team_stats:
            statistic = stat.get('aria-label')
            if statistic == 'team_name':
                continue
            value = stat.text
            data_dict[statistic] = value
        for analytic in team_analytics:
            statistic = analytic.get('aria-label')
            if statistic == 'team_name':
                continue
            value = analytic.text
            data_dict[statistic] = value
        writer.writerow(data_dict)
        time.sleep(10)

ValueError: dict contains fields not in fieldnames: None